In [8]:
import os
import torch
import sys

sys.path.append("/".join(os.getcwd().split("/")[:-1]))
from common import *

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_name = 'llama3-8b' #'opt-6.7b'
checkpoint_path = '../models/llama3-8b' #'../models/opt-6.7b'

/local/huzaifa/jupytervenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step1：Load Model

In [9]:
from transformers import OPTForCausalLM,AutoModelForCausalLM
from transformers import AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(checkpoint_path, device_map=device, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, device_map=device)

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.46it/s]


# Step2：Core Neurons similarity evaluation

In [ ]:
import torch
import re
import numpy as np
import matplotlib.pyplot as plt

# model.cuda()
token_sparsity = 0.2
sparsity = 0.4

def activation_similarity(sentence1,sentence2):
    data = [sentence1,sentence2]
    Layer_num = 25

    activations, SEN_F = get_sentence_core_neurons(model_name, model, tokenizer, data, Layer_num, device, token_sparsity, sparsity)

    similarity=len(np.intersect1d(SEN_F[0], SEN_F[1]))/len(SEN_F[1])
    return similarity




# Step4: Pearson Correlation Coefficient on STS-B

In [11]:
from datasets import load_dataset
dataset = load_dataset('mteb/stsbenchmark-sts')

print("Dataset Info:")
print(dataset)

Dataset Info:
DatasetDict({
    train: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 1379
    })
})


In [ ]:
sim_all=[]
true_all=[]
data=dataset['validation']

for i in range(len(data)):
    sentence1 = dataset['validation'][i]['sentence1']
    sentence2 = dataset['validation'][i]['sentence2']
    act_sim = activation_similarity(sentence1,sentence2)
    true_sim = dataset['validation'][i]['score']
    sim_all.append(act_sim)
    true_all.append(true_sim)
    

In [34]:
from scipy.stats import spearmanr

correlation, p_value = spearmanr(sim_all, true_all)
print(f"Spearman correlation: {correlation}")
print(f"P-value: {p_value}")

Spearman correlation: 0.5874475337292442
P-value: 7.283538395503928e-140


# Step4: Pearson Correlation Coefficient on Sick

In [5]:
from datasets import load_dataset
dataset = load_dataset( 'maximedb/sick_nl')

print("Dataset Info:")
print(dataset)

Dataset Info:
DatasetDict({
    train: Dataset({
        features: ['pair_ID', 'sentence_A', 'sentence_B', 'entailment_label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset', 'SemEval_set', 'label', 'label_seq2seq'],
        num_rows: 4439
    })
    validation: Dataset({
        features: ['pair_ID', 'sentence_A', 'sentence_B', 'entailment_label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset', 'SemEval_set', 'label', 'label_seq2seq'],
        num_rows: 495
    })
    test: Dataset({
        features: ['pair_ID', 'sentence_A', 'sentence_B', 'entailment_label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset', 'SemEval_set', 'label', 'label_seq2seq'],
        num_rows: 4906
    })
})


In [ ]:
sim_all=[]
true_all=[]
data=dataset['validation']

for i in range(len(data)):
    sentence1 = dataset['validation'][i]['sentence_A']
    sentence2 = dataset['validation'][i]['sentence_B']
    act_sim=activation_similarity(sentence1,sentence2)
    true_sim=dataset['validation'][i]['relatedness_score']
    sim_all.append(act_sim)
    true_all.append(true_sim)

In [8]:
from scipy.stats import spearmanr

correlation, p_value = spearmanr(sim_all, true_all)
print(f"Spearman correlation: {correlation}")
print(f"P-value: {p_value}")

Spearman correlation: 0.3683946806722181
P-value: 2.3418180334252915e-17
